In [102]:
!pip install boltons -q

In [103]:
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from boltons.iterutils import windowed
from tqdm import tqdm, tqdm_notebook
MAIN_PATH = '/Users/luyiming/Downloads/中文歌词语料/song_list_chinese.json'

In [104]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [105]:
song_data = json.loads(open(MAIN_PATH).read())
song_data = list(map(dict, song_data))
print(song_data[1]['lyric'])
print(type(song_data[1]))
print('song num : ', len(song_data))

我想我的脚步已不能停
要去山顶看很美很美的风景
我只是遗憾
再也不能和你去聆听
那支会孤单很久的风铃
抱歉 我不能带你去远行
如果想我了 就看天边最亮的星星
我只是遗憾
我的温暖你再也不能感应
最后唱首天使的翅膀给你听
相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你

抱歉 我不能带你去远行
如果想我了 就看天边最亮的星星
我只是遗憾
我的温暖你再也不能感应
最后唱首天使的翅膀给你听
相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你

相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你
相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你
今生今世 归去来兮
我会找个天使替我去爱你
<class 'dict'>
song num :  6828


In [106]:
jay = '林夕'
jay_song = []
for song in song_data:
    try:
        if song and jay in song.get('lyric', ''):
            lyric = song['lyric']
            
            # filter
            lyric = '\n'.join(filter(lambda x:x and ':' not in x and '：' not in x and '【' not in x, lyric.split('\n')))
            
            jay_song.append(lyric)
            print(lyric)
            print('-'*100)
    except:1
print('jay song num : ', len(jay_song))

我没有胆挂念 你没有心见面 试问我可以去边
只要我出现 只怕你不便 亦连累你丢脸
你是我的秘密 我是你的废物 缺席也不算损失
今晚你生日 祝我有今日 地球上快消失
眼泪还是留给天抚慰 你是前度 何必听我吠
再不走 有今生 无下世 你是否想我 起这个毒誓
宁愿失恋亦不想失礼 难道要对着你力歇声嘶
即使不抵 都要眼闭 我这种身世 有甚么资格 献世
我共你不够熟 眼泪也比较浊 也没气质对你哭
不介意孤独 比爱你舒服 别离就当祝福
眼泪还是留给天抚慰 你是前度 何必听我吠
再不走 有今生 无下世 你是否想我 起这个毒誓
宁愿失恋亦不想失礼 难道要对着你力歇声嘶
即使不抵 都要眼闭 我这种身世 有甚么资格 献世
即使不抵 都要眼闭
我自卑不怕 有自尊只怕 怕献世
----------------------------------------------------------------------------------------------------
分手时内疚的你一转脸
为日后不想有什么牵连
当我工作 睡觉 祷告 娱乐 那么刻意过好每天
谁料你见松绑了又愿见面
谁当初想摆脱被围绕左右
过后谁人被遥控于世界尽头
勒到呼吸困难才知变扯线木偶
这根线其实说到底 谁拿捏在手
不聚不散 只等你给另一对手擒获
那时青丝 不会用上余生来量度
但我拖着躯壳 发现沿途寻找的快乐
仍系于你肩膊 或是其实在等我舍割
然后断线风筝会直飞天国
这些年望你紧抱他出现
还凭何担心再互相纠缠
给我找个伴侣找到留下你的足印也可发展
全为你背影逼我步步向前
如一根丝牵引着拾荒之路
结在喉咙内痕痒得似有还无
为你安心我在微笑中想吐未吐
只想你和伴侣要好才顽强病好
不聚不散 只等你给另一对手擒获
以为青丝 不会用上余生来量度
但我拖着躯壳 发现沿途寻找的快乐
仍系于你肩膊 或是其实在等我舍割
然后断线风筝会直飞天国
一直不觉 捆绑我的未可扣紧承诺
满头青丝 想到白了仍懒得脱落
被你牵动思觉 最后谁愿缠绕到天国
然后撕裂躯壳 欲断难断在 不甘心去舍割
难道爱本身可爱在于束缚
无奈你我牵过手 没绳索
----------------------------------------------------------------------------------------------------
徘徊过多

In [107]:
data_config = {'seq_len':30, 'batch_size':64}
model_config = {'embedding_dim':128, 'hidden_dim':64, 'vocab_size':None}
train_config = {'epochs':30, 'lr':1e-3}
START = '<START>'
STOP = '<STOP>'

class Sequences(Dataset):
    
    def __init__(self, seq_len, raw_data):
        self.seq_len = seq_len
        self.raw_data = raw_data
        self.char2idx = {START:0, STOP:1}
        
        self.sequences = []
        for instance in tqdm_notebook(raw_data):
            for i in set(instance):
                if i not in self.char2idx:
                    self.char2idx[i] = len(self.char2idx)
            
            instance = ['<START>'] + list(instance) + ['<STOP>']
            for i in range(len(instance) - seq_len):
                self.sequences.append(instance[i:i + seq_len])
        
        self.sequences = np.array([[self.char2idx[i] for i in seq] for seq in self.sequences])       
        model_config['vocab_size'] = len(self.char2idx)
        
    def __getitem__(self, idx):
        return self.sequences[idx, :-1], self.sequences[idx, 1:]
    
    def __len__(self):
        return len(self.sequences)
        

In [108]:
dataset = Sequences(seq_len=data_config['seq_len'], raw_data=jay_song)
print(len(dataset))
train_loader = DataLoader(dataset, batch_size=data_config['batch_size'])


31283


In [109]:
print(dataset.char2idx)
print(dataset[0])

{'<START>': 0, '<STOP>': 1, '连': 2, '身': 3, '的': 4, '质': 5, '抵': 6, '有': 7, '宁': 8, '哭': 9, '么': 10, '给': 11, '缺': 12, '地': 13, '要': 14, '想': 15, '念': 16, '算': 17, '下': 18, '献': 19, '资': 20, '日': 21, '个': 22, '气': 23, '试': 24, '没': 25, '意': 26, '无': 27, '嘶': 28, '介': 29, '别': 30, '废': 31, '听': 32, '闭': 33, '怕': 34, '卑': 35, '\n': 36, '亦': 37, '否': 38, '我': 39, '脸': 40, '还': 41, '服': 42, '边': 43, '吠': 44, '甚': 45, '比': 46, '留': 47, '以': 48, '出': 49, '何': 50, '共': 51, '熟': 52, '礼': 53, '问': 54, '够': 55, ' ': 56, '可': 57, '眼': 58, '世': 59, '上': 60, '丢': 61, '使': 62, '祝': 63, '离': 64, '不': 65, '慰': 66, '当': 67, '去': 68, '也': 69, '舒': 70, '面': 71, '球': 72, '爱': 73, '声': 74, '快': 75, '难': 76, '格': 77, '见': 78, '前': 79, '力': 80, '种': 81, '席': 82, '较': 83, '天': 84, '失': 85, '愿': 86, '密': 87, '浊': 88, '胆': 89, '必': 90, '歇': 91, '独': 92, '你': 93, '只': 94, '抚': 95, '损': 96, '道': 97, '即': 98, '尊': 99, '都': 100, '自': 101, '心': 102, '毒': 103, '累': 104, '着': 105, '消': 106, '对': 107, '就': 108, '泪': 10

In [110]:
class RNN_LM(nn.Module):
    
    def __init__(self, hidden_dim, vocab_size, embedding_dim, device='cpu'):
        super(RNN_LM, self).__init__()
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.device = device
        
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.rnn = nn.GRU(self.embedding_dim, self.hidden_dim, batch_first=True)
        self.linear = nn.Linear(self.hidden_dim, self.vocab_size)
        
    def hidden(self, batch_size):
        return torch.randn(1, batch_size, self.hidden_dim).to(self.device)
    
    def forward(self, x, hidden):
        x_emb = self.embedding(x)
        x_rnn, hidden = self.rnn(x_emb.unsqueeze(1), hidden)
        x_out = self.linear(x_rnn.squeeze(1))
        return x_out, hidden

model = RNN_LM(hidden_dim=model_config['hidden_dim'],
            vocab_size=model_config['vocab_size'],
            embedding_dim=model_config['embedding_dim']).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=train_config['lr'],
)

RNN_LM(
  (embedding): Embedding(1693, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=1693, bias=True)
)


In [111]:
# train
model.train()
train_losses = []

for epoch in range(train_config['epochs']):
    progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, targets in progress_bar:
        batch_size = inputs.size(0)
        hidden = model.hidden(batch_size)

        model.zero_grad()
        
        loss = 0
        for char_idx in range(inputs.size(1)):
            output, hidden = model(inputs[:, char_idx].to(device), hidden)
            loss += criterion(output, targets[:, char_idx].to(device))

        loss.backward()

        optimizer.step()
        
        avg_loss = loss.item() / inputs.size(1)
        
        progress_bar.set_description(f'Loss: {avg_loss:.3f}')
        
        losses.append(avg_loss)
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')
    

Epoch #1	Train Loss: 6.077


Epoch #2	Train Loss: 5.283


Epoch #3	Train Loss: 4.778


Epoch #4	Train Loss: 4.370


Epoch #5	Train Loss: 4.039


...

Epoch #26	Train Loss: 1.729


Epoch #27	Train Loss: 1.689


Epoch #28	Train Loss: 1.651


Epoch #29	Train Loss: 1.617


Epoch #30	Train Loss: 1.586


In [116]:
model.eval()
char2idx = dataset.char2idx
idx2char = {v:k for k,v in char2idx.items()}

TOPK = 5
temperature = 1.0
seed = START
text = ''
with torch.no_grad():
    batch_size = 1
    hidden = model.hidden(batch_size)
    last_char = char2idx[seed]
    while len(text) < 300 or (len(text) > 300 and text[-1] == '\n'):
        output, hidden = model(torch.LongTensor([last_char]).to(device), hidden)
        
        distribution = output.squeeze().div(temperature).exp()
        
        # topk
        distribution_k, indice_k = torch.topk(distribution, TOPK)
        distribution_new = torch.zeros_like(distribution)
        for idx,val in zip(indice_k, distribution_k):
            distribution_new[idx] = val
        #print(distribution)
        
        guess = torch.multinomial(distribution_new, 1).item()
        
        last_char = guess
        cur_char = idx2char[guess]
        if cur_char == STOP:break
        
        text += cur_char
        
print(text)

你的情绪全是假的
和你半夜做 不管
在旁是谁为谁呼唤吻下去
只想风想再不放弃得太乖
是无力 或有你会有太好过
倘若那一起
不可以沉下去 这样浪漫的地烟面
只想去成全让我们可爱你去韬光
不是你
一边享受 就变黑暗
沉路忆非爱你是为了还好是 爱情地 当时的月亮
不知过不想再说话）
可能为下次
无端想到不可以暂时别太累孤独
如若我  不失为天大的幸福 将这世界比我大
不是我 不过可惜伤害
有你还好 一起来不苦
谁活得不要
小小明星
原来再见我的一切
只要是那么不好
因没有必要呵你似我们不能再见
如果说等待有多少工作
与我我 我暂时受 不在乎 才能对别说到出头这天 我都会有爱情
不想像个闹钟 按一封情
